<a href="https://colab.research.google.com/github/BojarLab/CandyCrunch/blob/main/CandyCrunch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/BojarLab/CandyCrunch/main/images/candycrunch_logo_light_banner_pt.svg" height="300" align="center" style="height:240px">

# **CandyCrunch**

This notebook runs [CandyCrunch](https://github.com/BojarLab/CandyCrunch/) on mzML and mzXML files. 
 
MS2 peak analysis is also available after molecule prediction using CandyCrumbs.  

### **What is CandyCrunch?**

CandyCrunch is a deep-learning model trained to predict the structure of glycans from an LC-MS/MS run. It requires at the minimum an input file and a glycan class from "O", "N", "lipid", "free", and "other".


### **User Guide**

- In order for this notebook to read any files on your Google Drive you will first need to right click on the folder containing this notebook and select 'Add a shortcut to Drive'.
- Make sure your input file is in one of the following formats: .mzML, .mzXML or .xlsx (already processed)
- **Step 2** (generating the annotations) should take between 1 and 5 minutes
- In **Step 3**, when saving the the annotations as a csv or pdf rememeber to always change the name or the previous file will be overwritten
- **Steps 1-2** need to have been executed in order for **Step 3 and Step 4** to work


In [ ]:
#@markdown # Step 1: Install all neccessary packages

from IPython.utils.io import capture_output
import os
import subprocess

from google.colab import drive
drive.mount('/content/drive')

try:
  with capture_output() as captured:
    !apt install libcairo2-dev
    !pip install CandyCrunch[draw]
    from glycowork.motif.draw import GlycoDraw
    from CandyCrunch.prediction import *
    from CandyCrunch.analysis import *
    !apt-get install wkhtmltopdf
    !pip install imgkit
    !pip install IPython~=7.34.0 
    from IPython.core.display import display,HTML
    !pip install imgkit==1.2.3
    import imgkit
    !pip install PIL~=8.4.0
    from PIL import Image
    import io
    import warnings
    import base64

except subprocess.CalledProcessError as captured:
  print(captured)
  raise

Mounted at /content/drive


In [ ]:
#@markdown # Step 2: Select your input file and predict structures  

#@markdown ### Enter the directory address to your input files
directory_address  = 'drive/MyDrive/example_folder/'  #@param {type:"string"}
#@markdown ### Enter the name of your input file (as one of the formats .mzML, .mzXML, or .xlsx)
filename = 'example_file.mzML'  #@param {type:"string"}
spectra_filepath = ''
if ".xlsx" in filename:
  spectra_filepath = directory_address + filename
else:
  if  "." in filename:
    filename = filename.split('.')[0]
  input_address = directory_address + filename
  for extension,extraction_function in [('.mzML',process_mzML_stack),('.mzXML',process_mzXML_stack)]:
    if os.path.isfile(input_address + extension):
      if not os.path.isfile(filename + extension + '.xlsx'):
        extraction_function(input_address + extension,intensity=True).to_excel(filename+extension+'.xlsx',index=False)
      spectra_filepath = filename + extension + '.xlsx'
      break

#@markdown ### Select the model parameters:
glycan_class = 'O' #@param ["O", "N", "lipid", "free","other"] 
mode = 'negative' #@param ["negative", "positive"]
modification = 'reduced' #@param ["reduced", "permethylated", "other/none"]
liquid_chromatography = 'PGC' #@param ["PGC", "C18", "other"]
trap = 'linear' #@param ["linear", "orbitrap", "amazon", "other"]

warnings.filterwarnings("ignore", category=RuntimeWarning)
df_out,spectra_out = wrap_inference(spectra_filepath, glycan_class,
                                    mode = mode, modification = modification, lc = liquid_chromatography, trap = trap,
                                    spectra=True,experimental=False,supplement=False)

glycan_pred_list = [x[0][0] if x else [] for x in df_out['predictions'].tolist()]
glycan_probs_list = [f'{round(x[0][1]*100,2)}%' if x else 'N/a' for x in df_out['predictions'].tolist()]
glycan_img_list = [GlycoDraw(x).as_svg() if x else '' for x in glycan_pred_list]

glycan_all_preds = []
for preds in df_out['predictions'].tolist():
  glycan_all_preds.append([x[0] for x in preds])
["<br>".join(x) for x in glycan_all_preds]
alt_preds = [x[1:] for x in glycan_all_preds]

glycan_all_probs = []
for preds in df_out['predictions'].tolist():
  glycan_all_probs.append([round(x[1]*100,2) for x in preds])
glycan_all_probs_string = [[str(x)+'%' for x in y] for y in glycan_all_probs]
["<br>".join(x) for x in glycan_all_probs_string]

display_df = df_out.reset_index()
display_df = display_df[[x for x in display_df.columns if x not in ['top_fragments','adduct','composition']]]
display_df['predictions'] = glycan_pred_list
display_df['predicted_snfg'] = glycan_img_list
display_df['prediction_probability'] = glycan_probs_list
display_df = display_df.rename(columns={"index": "m/z","predictions": "predicted_IUPAC",})
if 'rel_abundance' in df_out:
  display_df = display_df.rename(columns={"rel_abundance": "abundance"})
  display_df = display_df[['m/z', 'predicted_IUPAC', 'prediction_probability','num_spectra', 'charge', 'RT', 'abundance', 'predicted_snfg']]
  display_df['abundance'] = [round(x,2) for x in display_df['abundance'].tolist()]
else:
  display_df = display_df[['m/z', 'predicted_IUPAC', 'prediction_probability','num_spectra', 'charge', 'RT','predicted_snfg']]
display_df.index.name = 'prediction ID'
display_df['m/z'] = [round(x,2) for x in display_df['m/z'].tolist()]
display_df['predicted_IUPAC'] = ["<br>".join(x) for x in glycan_all_preds]
display_df['prediction_probability'] = ["<br>".join(x) for x in glycan_all_probs_string]
display_df['alternative_snfg'] = [[GlycoDraw(x).as_svg() for x in shot if GlycoDraw(x)] for shot in alt_preds]

def svg_to_base64(svg):
    svg_bytes = io.BytesIO(svg.encode())
    svg_data = svg_bytes.read()
    return 'data:image/svg+xml;base64,' + base64.b64encode(svg_data).decode('utf-8')

def format_name(name):
    return f'<span style="font-size: 16px">{name}</span>'

def format_main_image(main_image):
    return f'<div style="text-align: center"><img src="{svg_to_base64(main_image)}" /></div>'
  
def format_alt_images(alt_images):
    svg_list = []
    for alt_image in alt_images:
      svg_list.append(f'<div style="text-align: center; margin-bottom: 10px;"><img src="{svg_to_base64(alt_image)}" /></div>')
    return f''.join(svg_list)

def svg_list_to_html(svg_list):
    cell_html = ''
    for svg_file in svg_list:
        svg_base64 = svg_to_base64(svg_file)
        cell_html += f'<img src="data:image/svg+xml;base64,{svg_base64}" style="display:block; margin: 10px 0;">'
    return cell_html

format_dict = {'predictions':format_name, 'predicted_snfg': format_main_image, 'alternative_snfg': format_alt_images}

html_table = display_df.to_html(escape=False, formatters=format_dict)
html_table = html_table.replace('<th>', '<th style="font-size: 20px">')

new_style = """
<style>
table {
  border-collapse: collapse;
}

th {
  text-align: center;
  background-color: #f2f2f2;
  padding: 8px;
}

td {
  text-align: center;
  padding: 8px;
}

.image {
  text-align: center;
}

.image img {
  width: 200px;
  height: auto;
}

tr.start-page {
    page-break-before: always;
}
tr.end-page {
    page-break-after: always;
}
</style>
"""

html_table = new_style + html_table


# Display HTML table
display(HTML(html_table))

In [ ]:
#@markdown # Step 3a: Save your output as a csv table

#@markdown ### Enter the name of the csv to be saved:
csv_name = 'example_file'  #@param {type:"string"}

saving_df = display_df.copy(deep=True)
saving_df['secondary_IUPAC_predictions'] = alt_preds
saving_df['prediction_probability'] = [x[0] for x in glycan_all_probs]
saving_df['predicted_IUPAC'] = glycan_pred_list
saving_df = saving_df.drop(columns=['predicted_snfg','alternative_snfg'])
saving_df['secondary_prediction_probabilities'] = [x[1:] for x in glycan_all_probs]

print('Creating csv...') 
try:
  with capture_output() as captured:
    saving_df.to_csv(f'{directory_address}{csv_name}.csv')
except subprocess.CalledProcessError as captured:
  print(captured)
  raise
print(f'Saved csv in {directory_address}{csv_name}.csv')

In [ ]:
#@markdown # Step 3b: Save your output as a pdf with images

#@markdown ### Enter the name of the pdf to be saved:
pdf_name = 'example_file'  #@param {type:"string"}

print_style = """
<style>
table {
  border-collapse: collapse;
}

th {
  text-align: center;
  background-color: #f2f2f2;
  padding: 8px;
}

td {
  text-align: center;
  padding: 8px;
}

.image {
  text-align: center;
}

.image img {
  width: auto;
  height: auto;
}

tr.start-page {
    page-break-before: always;
}
tr.end-page {
    page-break-after: always;
}
</style>
"""


print('Creating pdf...') 
try:
  with capture_output() as captured:
    pages = []
    for i in range(0, len(display_df), 4):
        page_df = display_df.iloc[i:i+4]
        page_html = page_df.to_html(escape=False, formatters=format_dict, classes='table')
        page_html = page_html.replace('<th>', '<th style="font-size: 20px">')
        page_html = print_style + page_html
        pages.append(page_html)

    df_steps = [x for x in range(0,len(display_df),4)]
    images = [imgkit.from_string(x,False) for x in pages]
    pil_images = [Image.open(io.BytesIO(x)) for x in images]

    Image.open(io.BytesIO(images[0])).save(f'{directory_address}{pdf_name}.pdf', save_all=True, append_images=pil_images[1:])
except subprocess.CalledProcessError as captured:
  print(captured)
  raise
print(f'Saved pdf in {directory_address}{pdf_name}.pdf')

In [ ]:
#@markdown # Step 4a: Analyse the MS2 spectrum of a glycan predicted above

#@markdown ### Enter the **prediction ID** of the glycan prediction you wish to analyse:
ID = 4 #@param {type:"integer"}

#@markdown ### Enter the **mass threshold** to use when searching for fragments:
threshold = 1 #@param {type:"number"}

#@markdown ## Click and drag to zoom in on the spectrum. Hover over each bin to display the predicted MS2 frgment.

from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()

rows = []
parent_glycan = df_out.iloc[ID,0][0][0]
spectrum = sorted(spectra_out[ID].items())
masses = [x[0] for x in spectrum]
intensities = [x[1] for x in spectrum]
charge = display_df.at[ID,'charge']
main_annotations = CandyCrumbs(parent_glycan,masses,threshold,charge = charge)
sorted_masses = [x for x in main_annotations]
frags = [main_annotations[x]['Domon-Costello nomenclatures'][0] if main_annotations[x] else [] for x in main_annotations]
svgs_in = [GlycoDraw(domon_costello_to_fragIUPAC(parent_glycan,x)).as_svg() if x else '' for x in frags]
charges = [main_annotations[x]['Fragment charges'][0] if main_annotations[x] else [] for x in main_annotations] 
rows = []
for i,mass in enumerate(masses):
  row_dict = {'mass':mass,'intensity':intensities[i],'frag':domon_costello_to_html(frags[i]),'img':svgs_in[i], 'charge':charges[i]}
  rows.append(row_dict)
row_df = pd.DataFrame(rows)


output_file("foo.html")

TOOLS = "hover,save,box_zoom,reset,wheel_zoom"
TOOLTIPS = """
<div>
m/z: @mass{1.11}<br>
Domon Costello: @frag<br>
charge: @charge<br>
@img{safe}
</div>
"""
p = figure(plot_width=2000, plot_height=600, y_axis_label="normalised intensity",x_axis_label="m/z", tools=TOOLS, tooltips=TOOLTIPS)

p.vbar(x='mass', bottom=0, top='intensity', width=0.1, source=row_df)
hover = p.select(dict(type=HoverTool))
hover.mode='mouse'
show(p)
display(GlycoDraw(parent_glycan))
HTML('<br>'*3)

In [ ]:
#@markdown # Step 4b: Show more possible MS2 structures of a glycan predicted above

#@markdown ### Enter the **prediction ID** of the glycan prediction you wish to analyse:
ID = 7 #@param {type:"integer"}

#@markdown ### Enter the **mass threshold** to use when searching for fragments:
threshold = 1 #@param {type:"number"}

#@markdown ### Enter the **number of fragments** you wish to analyse:
topk_fragments = 15 #@param {type:"integer"}

parent_glycan = df_out.iloc[ID,0][0][0]
if '{' in parent_glycan:
  raise Exception('Unfortunately structures with undefined topologies are not currently supported.')
parent_image = GlycoDraw(parent_glycan).as_svg()
alt_preds = [x[0] for x in df_out.iloc[ID,0] if x[0]!= display_df.iloc[0,1]]
alt_pred_probs = [x[1] for x in df_out.iloc[ID,0] if x[0] in alt_preds]
main_fragments = {k:v for k, v in sorted(spectra_out[ID].items(), key = lambda item: item[1],reverse=True)}
main_peaks = [round(x,2) for x in list(main_fragments.keys())][:topk_fragments]
if len(main_peaks) == 0:
  raise Exception('Unfortunately no MS2 peaks were found.')
main_intensities = [x for x in list(main_fragments.values())][:topk_fragments]
idx = np.argsort(main_peaks)
main_peaks = [main_peaks[x] for x in idx]
main_intensities = [main_intensities[x] for x in idx]

try:
  all_annotations = CandyCrumbs(parent_glycan,main_peaks,threshold,simplify=False,charge = display_df.at[ID,'charge'])
  primary_annotations = CandyCrumbs(parent_glycan,main_peaks,threshold,simplify=True,charge = display_df.at[ID,'charge'])
except:
  print('Unfortunately the MS2 fragments for this structure cannot be calculated, please try another.')

frag_DC_list = []
frags_img_list = []
for x in all_annotations:
  if all_annotations[x]:
    dc_annotations = all_annotations[x]['Domon-Costello nomenclatures']
    primary_annotation = primary_annotations[x]['Domon-Costello nomenclatures'][0]
    dc_annotations.sort(key=len)
    dc_annotations.insert(0, dc_annotations.pop(dc_annotations.index(primary_annotation)))
    dc_names = "<br>".join([domon_costello_to_html(frag) for frag in dc_annotations])
    frag_DC_list.append('<center>'+dc_names+'</center>')
  else:
    dc_annotations = []
    frag_DC_list.append('<center>'+'No fragments found'+'</center>')
  frag_IUPAC_list = [domon_costello_to_fragIUPAC(parent_glycan,frag) for frag in dc_annotations]
  frags_img_list.append([GlycoDraw(x).as_svg() for x in frag_IUPAC_list])
  # [[GlycoDraw(x).as_svg() for x in shot if GlycoDraw(x)] for shot in alt_preds]
frag_table = pd.DataFrame(columns=['fragment_mass','normalised_intensity','possible_annotations','possible_snfg_annotations'])
frag_table['fragment_mass'] = main_peaks
main_intensities = [round(x,2) for x in main_intensities]
frag_table['normalised_intensity'] = main_intensities
frag_table['possible_annotations'] = frag_DC_list
frag_table['possible_snfg_annotations'] = frags_img_list
frag_table = frag_table.sort_values('normalised_intensity',ascending=False)
frag_table[[x for x in frag_table.columns if x!= 'possible_snfg_annotations']] = frag_table[[x for x in frag_table.columns if x != 'possible_snfg_annotations']].astype(str) 

MS2_format_dict = {'fragment_mass':format_name,'normalised_intensity':format_name,'possible_annotations':format_name,'possible_snfg_annotations': format_alt_images}

frag_table = '<center>'+parent_image+"<br>"+frag_table.to_html(escape=False, formatters=MS2_format_dict)
frag_table = frag_table.replace('<th>', '<th style="font-size: 20px">')
frag_table = new_style + frag_table


# Display HTML table
display(HTML(frag_table))